<a href="https://colab.research.google.com/github/detektor777/colab_list/blob/main/whisper_upload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Перейити в гугл диск и загрузить в корень диска файл

https://drive.google.com/drive/

In [ ]:
#@title ##**Имя видео файла** { display-mode: "form" }
file_name = input("Введите имя файла: ") or "input.mp3"

In [ ]:
#@title ##**Установить зависимости** { display-mode: "form" }
!git clone https://huggingface.co/spaces/openai/whisper
%cd whisper
!pip install -r requirements.txt
!pip install gradio
!pip install pysrt
import pysrt

In [ ]:
#@title ##**Подключение к гугл диску** { display-mode: "form" }
from google.colab import drive
drive.mount('/mnt/gdrive')


In [ ]:
#@title ##**Разбить файл** { display-mode: "form" }
from pydub import AudioSegment
from pydub.utils import make_chunks
import os
import shutil

split = "30_second" #@param ["30_second","auto"]
chunk_folder = "/content/whisper/audio_chunks"
if os.path.isdir(chunk_folder):
    shutil.rmtree(chunk_folder)

if (split == "30_second"):

  path = '/mnt/gdrive/MyDrive/'
  full_path = os.path.join(path, file_name)

  def split_audio_file(audio_file_path, chunk_length_ms=30000):

      audio = AudioSegment.from_file(audio_file_path)
      audio_chunks = make_chunks(audio, chunk_length_ms)

      os.makedirs(chunk_folder, exist_ok=True)

      for i, chunk in enumerate(audio_chunks):
          chunk_name = f'{chunk_folder}/chunk_{i:04d}.wav'
          print(chunk_name)
          chunk.export(chunk_name, format="wav")

      return chunk_folder

  split_audio_file(full_path)

if (split == "auto"):
  from librosa.util.utils import normalize
  import os
  import librosa
  import numpy as np
  import soundfile as sf
  import shutil
  import audioread

  import scipy.io.wavfile as wav
  import scipy.signal as signal
  from scipy.signal import find_peaks
  from scipy.io import wavfile
  import moviepy.editor as mp
  from scipy.io import loadmat


  chunk_folder = "/content/whisper/audio_chunks"
  if os.path.isdir(chunk_folder):
      shutil.rmtree(chunk_folder)
  os.makedirs(chunk_folder)
  path = '/mnt/gdrive/MyDrive/'

  full_path = os.path.join(path, file_name)

  def convert_to_wav(input_file):
      file_name, ext = os.path.splitext(input_file)
      if ext == '.wav':
          return input_file

      output_file = file_name + '.wav'
      sound = AudioSegment.from_file(input_file, format=ext[1:])
      sound.export(output_file, format='wav')
      return output_file

  full_path = convert_to_wav(full_path)
  print(full_path)
  # Загрузить аудиофайл и получить данные о сигнале
  rate, data = wavfile.read(full_path)

  # Нормализовать аудио до -3 дБ
  target_db = -3.0
  max_val = np.iinfo(data.dtype).max
  target_val = np.power(10.0, target_db / 20.0) * max_val
  data = data.astype(np.float64) * target_val / np.max(np.abs(data))
  data = data.astype(np.int16)

  # Сохранить нормализованный аудиофайл
  normalize_file = f"/mnt/gdrive/MyDrive/normalized_{file_name}"
  wavfile.write(normalize_file, rate, data)

  def split_chunk(input_file, output_dir, intervals_ms):
      # Открыть аудиофайл и загрузить его содержимое и частоту дискретизации
      audio, sr = sf.read(input_file)

      # Разделить аудиофайл на отдельные части
      for i in range(len(intervals_ms) - 1):
          start_ms = intervals_ms[i]
          end_ms = intervals_ms[i+1]

          # Вычислить начальный и конечный индексы в сэмплах
          start = int(start_ms / 1000 * sr)
          end = int(end_ms / 1000 * sr)

          # Записать отдельную часть аудиофайла в новый файл
          output_file = f'{chunk_folder}/chunk_{i:04d}.wav'
          sf.write(output_file, audio[start:end], sr)


  def split_audio(input_file, output_dir, min_duration=1.0, max_duration=10.0, silence_threshold=0.5, min_length=5):
      # Загрузить аудиофайл и получить данные о сигнале
      audio_file, sr = librosa.load(input_file, sr=None, mono=True)

      # Рассчитать длительность кадра и размер кадра
      frame_duration = 0.025
      frame_len = int(frame_duration * sr)

      # Рассчитать пороговое значение сигнала энергии для определения тишины
      peak_threshold = np.max(audio_file) * silence_threshold

      # Рассчитать сигнал энергии
      audio_file = np.concatenate((audio_file, np.zeros(frame_len - len(audio_file) % frame_len)))
      energies = np.sum(np.square(audio_file.reshape(-1, frame_len)), axis=1)

      # Найти пики в сигнале энергии
      peak_indices, _ = find_peaks(energies, height=peak_threshold)

      # Создать массив интервалов тишины
      silence_intervals = []

      # Если первый пик не в начале сигнала, добавить интервал от начала до первого пика
      if peak_indices[0] != 0:
          silence_intervals.append((0, peak_indices[0] * frame_len))

      # Добавить интервалы тишины между пиками, если длина интервала больше чем min_duration
      for i in range(len(peak_indices) - 1):
          start = peak_indices[i] * frame_len
          end = peak_indices[i+1] * frame_len
          duration = (end - start) / sr
          if duration > min_duration:
              silence_intervals.append((start, end))

      # Добавить последний интервал тишины, если его длина больше чем min_duration и меньше или равна max_duration
      if peak_indices[-1] * frame_len != len(audio_file) - 1:
          start = peak_indices[-1] * frame_len
          end = len(audio_file) - 1
          duration = (end - start) / sr
          if duration > min_duration and duration <= max_duration:
              silence_intervals.append((start, end))

      silence_intervals = [(start * 1000 / sr, end * 1000 / sr) for start, end in silence_intervals]

      means = []
      for start, end in silence_intervals:
          mean = (start + end) / 2
          means.append(mean)
      # вычислить длину аудиофайла в миллисекундах и добавить ее в конец списка intervals_ms
      file_duration_ms = np.round(len(audio_file) / sr * 1000).astype(int)
      means = np.append(means, file_duration_ms)
      means = np.insert(means, 0, 0)
      means = means.astype(int)
      i = 0
      min_interval = 15000
      max_interval = 30000
      while i < len(means) - 1:
          # проверить, если следующий элемент меньше чем текущий элемент + 15
          if means[i+1] < means[i] + min_interval:
              # удалить следующий элемент
              means = np.delete(means, i+1)
          # проверить, если следующий элемент равен текущему элементу + 15 и меньше чем текущий элемент + 30
          elif means[i+1] == means[i] + min_interval and means[i+1] < means[i] + max_interval:
              # оставить следующий элемент и перейти к следующей итерации
              i += 1
          # проверить, если следующий элемент больше чем текущий элемент + 30
          elif means[i+1] > means[i] + max_interval:
              # добавить новый элемент, который будет равен текущему + 30 и перейти к следующей итерации
              means = np.insert(means, i+1, means[i]+max_interval)
              i += 1
          else:
              i += 1
      means = np.append(means, file_duration_ms)
      print(silence_intervals)
      print(means)
      split_chunk(full_path, chunk_folder, means)
      return means

  time_labels = split_audio(normalize_file, chunk_folder)

In [ ]:
#@title ##**Установка модели** { display-mode: "form" }
import os
os.system("pip install git+https://github.com/openai/whisper.git")
import gradio as gr
import whisper

from share_btn import community_icon_html, loading_icon_html, share_js

model = whisper.load_model("large-v2")

 


In [ ]:
#@title ##**Запуск** { display-mode: "form" }
import os

import pysrt

def create_subtitles(start_time, end_time, text):
    subs = pysrt.SubRipFile()
    subs.append(pysrt.SubRipItem(index=1, start=start_time, end=end_time, text=text))
    return subs
    
def inference(audio):
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)
    
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    
    _, probs = model.detect_language(mel)
    
    options = whisper.DecodingOptions(fp16 = False)
    result = whisper.decode(model, mel, options)
    
    print(result.text)
    return result.text, gr.update(visible=True), gr.update(visible=True), gr.update(visible=True)

# Обработка каждого куска и запись результата в файл
result_text = ""
chunk_folder = "/content/whisper/audio_chunks"
chunk_files = sorted(os.listdir(chunk_folder))
for i in range(len(chunk_files)):
    if file_name.endswith('.wav'):
        audio_file = chunk_files[i]
        file_path = os.path.join(chunk_folder, audio_file)
        text = inference(file_path)[0]
        result_text += text

        start_time = time_labels[i]
        end_time = time_labels[i+1] if i+1<len(time_labels) else start_time + 5000
        subs = create_subtitles(start_time, end_time, text)
        result_subs.extend(subs)

# Сохранение результата в файл
with open('result.txt', 'w') as f:
    f.write(result_text)
result_subs.save("subtitles.srt")

In [ ]:
import re

with open('result.txt', 'r') as f:
    text = f.read()

# заменяем конец предложения на перенос строки
text = re.sub(r'([.?!])\s+', r'\1\n', text)

with open('result.txt', 'w') as f:
    f.write(text)
    
#@title ##**Скачать результат** { display-mode: "form" }
from google.colab import files

# загрузка файла result.txt в Colab
files.download('result.txt')